In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('/kaggle/input/demand-forecasting-kernels-only/train.csv') # 2013 01 - 2017 12
test = pd.read_csv('/kaggle/input/demand-forecasting-kernels-only/test.csv')  # 2018 01-03


def dmy_sep_dayofweek(df):
    df['date'] = pd.to_datetime(df.date, format="%Y-%m-%d")
    df['day'] = df.date.dt.day
    df['month'] = df.date.dt.month
    df['year'] = df.date.dt.year
    df['day_of_week'] = df.date.dt.dayofweek  # Mon:0, Sun: 6
    return df
    
train = dmy_sep_dayofweek(train)
test = dmy_sep_dayofweek(test)

#plt.figure(figsize=(25,25))
#mask = np.triu(np.ones_like(train.corr()))
#sns.heatmap(train.corr(), vmin=-1, vmax=1, mask=mask, annot=True, cmap='BrBG')

# day of week>year>month>item


#for var in ['month', 'year', 'day_of_week', 'item']:
#    grouped_sales = train.groupby(var).sales.mean()
#    sns.lineplot(x=train[var].value_counts().index.sort_values(), y=grouped_sales)
#    plt.xlabel(var)
#    plt.show()

#for year in range(2013,2018):
#    monthly_sales = train.loc[train.year == year].groupby('month').sales.mean()
#    sns.lineplot(x=range(1,13), y=monthly_sales, label=year)
#    plt.xlabel('month')
    
# item-store relationship
#plt.figure(figsize=(20,15))
#for store in range(1,11):
#    sales_byItem = train.loc[train.store==store].groupby('item').sales.mean()
#    sns.lineplot(x=range(1,51), y=sales_byItem, label=store
    
# exact same trend, seems fabricated


# relative sales ('smoother' to predict)
# Yearly growth pattern -> predict the next pt in curve
sales_avg = train.sales.mean()
rel_sales_byYear = pd.pivot_table(train, index='year', values='sales', aggfunc=np.mean)
rel_sales_byYear /= sales_avg

rel_sales_year = rel_sales_byYear.values.squeeze()
# try 2 ways of fitting
# fit return coefficients, 1d make them polynomial
fitdeg2 = np.poly1d(np.polyfit(range(2013,2018), rel_sales_year, 2))  # deg 3 too high, 4 would completely goes off

#plt.figure(figsize=(10,8))
#plt.plot(range(2013,2018), rel_sales_year, 'bo')
#plt.plot(range(2013,2019), fitdeg2(range(2013,2019)))
#plt.ylabel("Relative Sales")
#plt.xlabel("Year")

rel_sales2018 = round(fitdeg2(2018), 5)

# instead of prediction, multiply pass sales by factors
# 1. base sales day of week-item
# 2. day of week>year>month>item factors
base_sales_ref = pd.pivot_table(train, index='day_of_week', columns='item', values='sales',aggfunc=np.mean)
monthly_relSales = pd.pivot_table(train, index='month', values='sales',aggfunc=np.mean)
monthly_relSales.sales /= sales_avg
store_relSales = pd.pivot_table(train, index='store', values='sales',aggfunc=np.mean)
store_relSales.sales /= sales_avg

submission = pd.DataFrame(test['id'])
submission['sales'] = np.nan

for _, row in test.iterrows():
    dayofweek, month, year, item, store = row.day_of_week, row.month, row.year, row['item'], row.store
    base_sales = base_sales_ref.loc[dayofweek, item]
    monthFactor = monthly_relSales.loc[month]
    storeFactor = store_relSales.loc[store]
    pred_sales = base_sales * monthFactor * storeFactor * rel_sales2018
    submission.at[row['id'], 'sales'] = pred_sales
    

submission['sales'] = np.round(submission['sales']).astype(int)
submission.to_csv('submission.csv', index=False)